In [1]:
# put in firestore
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account.
cred = credentials.Certificate('atopile-880ca67acfe2.json')

app = firebase_admin.initialize_app(cred)

db = firestore.client()

# Function to add data to Firestore with the 'JLCPCB Part #' as the document name
def add_to_firestore(collection_name, data_dict):
    # Extract the 'JLCPCB Part #' to use as the document ID
    doc_id = data_dict.get('JLCPCB Part #')
    if not doc_id:
        raise ValueError("Missing 'JLCPCB Part #' in the data dictionary")

    # If 'Price' is already a list of dictionaries, no transformation is needed
    # If it's not, check if it's in the expected format (list of lists) and transform it
    if 'Price' in data_dict:
        if all(isinstance(p, dict) for p in data_dict['Price']):
            # If all elements are dictionaries, assume the format is correct
            pass
        else:
            # Transform nested arrays if necessary (e.g., 'Price' field)
            new_price_list = []
            for p in data_dict['Price']:
                # Ensure 'p' is a list or tuple with at least two elements
                if isinstance(p, (list, tuple)) and len(p) >= 2:
                    new_price_list.append({'quantity': p[0], 'price': p[1]})
                else:
                    raise ValueError(f"Invalid price format: {p}")
            data_dict['Price'] = new_price_list

    # Specify the document ID and set the data
    collection_ref = db.collection(collection_name).document(doc_id)
    collection_ref.set(data_dict)




In [20]:
import re
def parse_to_dict(component_string):
    # Regular expression patterns for each component
    patterns = {
        'current': r"(\d+\s*m?A)",
        'power_dissipation': r"(\d+\s*m?W)",
        'drain_source_voltage': r"(\d+\s*m?V)",
        'on_resistance': r"(\d+\.?\d*\s*m?Ω)(?=@)",
        'type': r"([NP] Channel)"
    }

    parsed_dict = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, component_string)
        if match:
            value = match.group().strip()
            # Convert milli units to base units if necessary and remove units for numerical values
            if key not in ['type']:
                numeric_value = re.match(r"\d+\.?\d*", value)
                if numeric_value:
                    numeric_value = numeric_value.group()
                    if 'm' in value:
                        # Remove the 'm' and convert the value
                        value = str(float(numeric_value) / 1000)
                    else:
                        # Use the numeric value directly
                        value = numeric_value

                # Add units to the key for numerical values
                unit = ('A' if key == 'current' else
                        'W' if key == 'power_dissipation' else
                        'V' if key == 'drain_source_voltage' else
                        'Ω' if key == 'on_resistance' else '')
                parsed_dict[f'{key}({unit})'] = value
            else:
                # For non-numerical values like 'type', just use the value as is
                parsed_dict[key] = value

    return parsed_dict

# Test the updated function with the new example string
new_example_string = "500V 28A 310W 155mΩ@10V,14A N Channel TO-3P  MOSFETs ROHS"
new_parsed_dict = parse_to_dict(new_example_string)
new_parsed_dict


# Test the function with the provided example string
example_string = "60V 115mA 200mW 7.5Ω@5V,50mA N Channel SOT-523(SC-75)  MOSFETs ROHS"
parsed_dict = parse_to_dict(example_string)
parsed_dict



{'current(A)': '0.115',
 'power_dissipation(W)': '0.2',
 'drain_source_voltage(V)': '60',
 'on_resistance(Ω)': '7.5',
 'type': 'N Channel'}

In [30]:
docs = db.collection('unparsed-parts').stream()

In [31]:
mosfets = []
for doc in docs:
    description = doc.to_dict().get('Description', '')
    if 'MOSFET' in description:  # This is case-sensitive
        mosfets.append(doc.to_dict())

In [22]:
# len(mosfets)

# take the first mosfet, parse it, and put it in firestore
mosfet = mosfets[0]

mosfet['Description']

parsed_dict = parse_to_dict(mosfet['Description'])

In [23]:
# combine the parsed dict with the original dict the key should be the jlcpcb part number

mosfet.update(parsed_dict)
mosfet

{'Assembly Type': 'Wave SolderingA PCB assembly fixture is needed to protect and support this part during the assembly process.',
 'Stock': 0,
 'Price': [{'quantity': '1+', 'price': 5.085},
  {'quantity': '10+', 'price': 4.449},
  {'quantity': '30+', 'price': 3.7035},
  {'quantity': '90+', 'price': 3.321},
  {'quantity': '450+', 'price': 3.1455},
  {'quantity': '900+', 'price': 3.066}],
 'MFR.Part #': 'FDA28N50',
 'Description': '500V 28A 310W 155mΩ@10V,14A N Channel TO-3P  MOSFETs ROHS',
 'Manufacturer': 'onsemi',
 'CAD Model': 'PCB Footprint or Symbol',
 'Datasheet': 'Download',
 'Package': 'TO-3P-3',
 'JLCPCB Part #': 'C105685',
 'Source': 'JLCPCB',
 'current(A)': '28',
 'power_dissipation(W)': '310',
 'drain_source_voltage(V)': '500',
 'type': 'N Channel',
 'on_resistance(Ω)': '0.155'}

In [28]:
# stick it in firestore components collection
add_to_firestore('mosfets', mosfet)

In [36]:
for component in mosfets:
    # Assume 'description' is the field with the string to parse
    description = component.get('Description', '')
    parsed_data = parse_to_dict(description)

    # check that all the required fields are present
    required_fields = ['current', 'power_dissipation', 'drain_source_voltage', 'on_resistance', 'type']
    for field in required_fields:
        if field not in parsed_data:
            # skip this component if any of the required fields are missing
            print(f"Skipping {component.get('JLCPCB Part #')} because '{field}' is missing")
            continue

    # Append the parsed data to the component dictionary
    component.update(parsed_data)

    # Use the custom function to add the updated document to Firestore
    try:
        add_to_firestore('mosfets', component)
        print(f"Successfully added: {component.get('JLCPCB Part #')}")
    except ValueError as e:
        print(f"Error: {e}")

Skipping C105685 because 'current' is missing
Skipping C105685 because 'power_dissipation' is missing
Skipping C105685 because 'drain_source_voltage' is missing
Skipping C105685 because 'on_resistance' is missing
Successfully added: C105685
Skipping C10644 because 'current' is missing
Skipping C10644 because 'power_dissipation' is missing
Skipping C10644 because 'drain_source_voltage' is missing
Skipping C10644 because 'on_resistance' is missing
Skipping C10644 because 'type' is missing
Successfully added: C10644
Skipping C10647 because 'current' is missing
Skipping C10647 because 'power_dissipation' is missing
Skipping C10647 because 'drain_source_voltage' is missing
Skipping C10647 because 'on_resistance' is missing
Successfully added: C10647
Skipping C10650 because 'current' is missing
Skipping C10650 because 'power_dissipation' is missing
Skipping C10650 because 'drain_source_voltage' is missing
Skipping C10650 because 'on_resistance' is missing
Successfully added: C10650
Skipping 

In [48]:
# retrieving footprints from jlcpcb
import easyeda2kicad
import subprocess

component_id = 'C114358'

command = [
    "easyeda2kicad",
    "--full",
    f"--lcsc_id={component_id}",
    f"--output=/Users/narayanpowderly/Documents/atopile-workspace/package-manager/temp/lib.kicad_sym",
    "--overwrite",
]
result = subprocess.run(command, capture_output=True, text=True)
print(result)



CompletedProcess(args=['easyeda2kicad', '--full', '--lcsc_id=C114358', '--output=/Users/narayanpowderly/Documents/atopile-workspace/package-manager/temp/lib.kicad_sym', '--overwrite'], returncode=0, stdout='-- easyeda2kicad.py v0.6.6 --\n', stderr='[INFO] Create lib.pretty footprint folder in /Users/narayanpowderly/Documents/atopile-workspace/package-manager/temp\n[INFO] Create lib.3dshapes 3D model folder in /Users/narayanpowderly/Documents/atopile-workspace/package-manager/temp\n[INFO] Create lib.kicad_sym symbol lib in /Users/narayanpowderly/Documents/atopile-workspace/package-manager/temp\n[INFO] Created Kicad symbol for ID : C114358\n       Symbol name : FS310LF-A\n       Library path : /Users/narayanpowderly/Documents/atopile-workspace/package-manager/temp/lib.kicad_sym\n[INFO] Created Kicad footprint for ID: C114358\n       Footprint name: TO-92SP-4_L5.3-W1.6-P1.27-L\n       Footprint path: /Users/narayanpowderly/Documents/atopile-workspace/package-manager/temp/lib.pretty/TO-92S

In [65]:
# retrieving footprints from jlcpcb
import easyeda2kicad
import subprocess

def get_footprint(component_id, dir):
    command = [
        "easyeda2kicad",
        "--full",
        f"--lcsc_id={component_id}",
        f"--output={dir}/lib",
        "--overwrite",
        "--ato",
        f"--ato_file_path={dir}/",
    ]
    result = subprocess.run(command, capture_output=True, text=True)
    # print(result)


In [55]:
get_footprint('C141541')

In [62]:
def parse_mapping(file_content):
    mapping = {'source': [], 'gate': [], 'drain': []}
    for line in file_content.split('\n'):
        line = line.strip()
        if 'S ~ pin' in line:
            pin_num = line.split()[-1]
            mapping['source'].append(pin_num)
        elif 'G ~ pin' in line:
            pin_num = line.split()[-1]
            mapping['gate'].append(pin_num)
        elif 'D ~ pin' in line:
            pin_num = line.split()[-1]
            mapping['drain'].append(pin_num)
    return mapping

def modify_footprint(file_content, mapping):
    lines = file_content.split('\n')
    modified_lines = []
    for line in lines:
        if 'pad' in line and 'smd' in line:
            parts = line.split()
            pad_num = parts[1]  # Original pad number
            new_pad_num = None
            # Determine the new pad number based on the SOT-23 standard
            if pad_num in mapping['gate']:
                new_pad_num = '1'
            elif pad_num in mapping['source']:
                new_pad_num = '2'
            elif pad_num in mapping['drain']:
                new_pad_num = '3'
            # Replace the pad number in the line
            if new_pad_num:
                parts[1] = new_pad_num
                line = '\t' + ' '.join(parts)  # Add a tab at the start of the modified line
        modified_lines.append(line)
    return '\n'.join(modified_lines)


def standardize_pin_map(ato_file, footprint_file):
    with open(ato_file, 'r') as f:
        file_content = f.read()

    mapping = parse_mapping(file_content)

    with open(footprint_file, 'r') as f:
        footprint_content = f.read()

    modified_footprint = modify_footprint(footprint_content, mapping)

    return modified_footprint

In [69]:

fets = db.collection('mosfets').stream()

mosfets = []
for doc in fets:
    description = doc.to_dict().get('Description', '')
    # if 'MOSFET' in description:  # This is case-sensitive
    mosfets.append(doc.to_dict())

In [83]:
# for each fet, get the footprint, modify it, and put it in firestore as a new field 'footprint'
import glob
import logging as log
dir = '/Users/narayanpowderly/Documents/atopile-workspace/package-manager/temp/'

for fet in mosfets:
    get_footprint(fet.get('JLCPCB Part #'))
    # find the ato file with the .ato extension, we dont know the name of the file, use glob
    try:
        ato_file = glob.glob(f"{dir}/*.ato")[0]
        footprint_file = glob.glob(f"{dir}/lib.pretty/*.kicad_mod")[0]
    except:
        print(f"Could not find ato file for {fet.get('JLCPCB Part #')}")
        continue


    # modify the footprint file
    modified_footprint = standardize_pin_map(ato_file, footprint_file)

    # add the modified footprint to the fet dict
    fet['footprint'] = modified_footprint
    # add the fet to firestore
    add_to_firestore('mosfets', fet)
    # delete the contents of the temp directory
    import os
    import shutil
    shutil.rmtree(dir)
    os.mkdir(dir)
    print(f"Added footprint to {fet.get('JLCPCB Part #')}")

Added footprint to C105685
Added footprint to C10644
Added footprint to C10647
Added footprint to C10650
Added footprint to C10654
Added footprint to C10657
Added footprint to C10663
Added footprint to C10666
Added footprint to C10668
Added footprint to C10669
Added footprint to C106992
Added footprint to C107589
Added footprint to C108891
Added footprint to C109445
Added footprint to C110100
Added footprint to C110101
Added footprint to C110158
Added footprint to C110710
Added footprint to C110711
Added footprint to C110712
Added footprint to C110713
Added footprint to C110715
Added footprint to C110716
Added footprint to C110717
Added footprint to C110718
Added footprint to C110724
Added footprint to C110725
Added footprint to C111356
Added footprint to C111970
Added footprint to C111971
Added footprint to C111980
Added footprint to C112654
Could not find ato file for C112722
Added footprint to C113315
Added footprint to C115168
Added footprint to C115225
Added footprint to C115827
A

In [84]:
# print mosfet[]
print(mosfets[0])

{'Assembly Type': 'Wave SolderingA PCB assembly fixture is needed to protect and support this part during the assembly process.', 'Stock': 0, 'type': 'N Channel', 'power_dissipation(W)': '310', 'Description': '500V 28A 310W 155mΩ@10V,14A N Channel TO-3P  MOSFETs ROHS', 'Price': [{'quantity': '1+', 'price': 5.085}, {'quantity': '10+', 'price': 4.449}, {'quantity': '30+', 'price': 3.7035}, {'quantity': '90+', 'price': 3.321}, {'quantity': '450+', 'price': 3.1455}, {'quantity': '900+', 'price': 3.066}], 'on_resistance(Ω)': '0.155', 'CAD Model': 'PCB Footprint or Symbol', 'current(A)': '28', 'Package': 'TO-3P-3', 'drain_source_voltage(V)': '500', 'Source': 'JLCPCB', 'JLCPCB Part #': 'C105685', 'Datasheet': 'Download', 'MFR.Part #': 'FDA28N50', 'Manufacturer': 'onsemi', 'footprint': '(module easyeda2kicad:TO-220IS_L10.16-W4.7-P2.54-L (layer F.Cu) (tedit 5DC5F6A4)\n\t(attr smd)\n\t(fp_text reference REF** (at 0 -4.0) (layer F.SilkS)\n\t\t(effects (font (size 1 1) (thickness 0.15)))\n\t)\n\t(

In [123]:
from firebase_admin import firestore

db = firestore.client()

# Updated field names
specified_current = 10  # Example value
specified_voltage = 30  # Example value
specified_rds_on = 1.5  # Example value
specified_type = 'N Channel'  # or 'P Channel'

# Step 1: Fetch MOSFETs with current >= 1.5x specified current
current_mosfets = db.collection("mosfets").where("current_A", ">", "30")

# Step 2: Filter MOSFETs with voltage >= specified voltage
voltage_mosfets = db.collection("mosfets").where("drain_source_voltage_V", ">=", "30")

# Step 3: Filter MOSFETs with Rds(on) <= specified Rds(on)
rds_on_mosfets = db.collection("mosfets").where("on_resistance_ohms", "<=", "1.5")

# Step 4: Filter MOSFETs with type == specified type
type_mosfets = db.collection("mosfets").where("type", "==", "N Channel")

# Step 5: Fetch the documents


def get_mosfets(specified_current, specified_voltage, specified_rds_on, specified_type):
    # Step 1: Fetch MOSFETs with current >= 1.5x specified current
    current_mosfets = db.collection("mosfets").where("current_A", ">", "30")

    # Step 2: Filter MOSFETs with voltage >= specified voltage
    voltage_mosfets = db.collection("mosfets").where("drain_source_voltage_V", ">=", "30")

    # Step 3: Filter MOSFETs with Rds(on) <= specified Rds(on)
    rds_on_mosfets = db.collection("mosfets").where("on_resistance_ohms", "<=", "1.5")

    # Step 4: Filter MOSFETs with type == specified type
    type_mosfets = db.collection("mosfets").where("type", "==", "N Channel")

    # find the intersection of the mosfets
    current_mosfets = set([doc.id for doc in current_mosfets.stream()])
    voltage_mosfets = set([doc.id for doc in voltage_mosfets.stream()])
    rds_on_mosfets = set([doc.id for doc in rds_on_mosfets.stream()])
    type_mosfets = set([doc.id for doc in type_mosfets.stream()])
    mosfets = current_mosfets.intersection(voltage_mosfets, rds_on_mosfets, type_mosfets)

    return mosfets

results = get_mosfets(specified_current, specified_voltage, specified_rds_on, specified_type))


/Users/narayanpowderly/Documents/atopile-workspace/.venv/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:295: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


{'C22162', 'C171516', 'C181090', 'C2648', 'C143712', 'C9214', 'C144931', 'C171514', 'C10657', 'C84143', 'C153779', 'C143688', 'C130365', 'C148148', 'C3010', 'C62725', 'C130980', 'C115840', 'C171532', 'C108891', 'C165232', 'C143704', 'C169754', 'C98736', 'C112722', 'C143679', 'C2720', 'C184884', 'C165276', 'C153802', 'C184823', 'C2671', 'C48657', 'C184834', 'C2619', 'C2617', 'C156306', 'C2670', 'C145546', 'C179750', 'C17098', 'C153226', 'C3001', 'C165225', 'C148153', 'C15791', 'C145537', 'C82938', 'C2710', 'C7617', 'C141823', 'C144927', 'C58228', 'C2711', 'C115829', 'C156338', 'C2927', 'C110716', 'C179753', 'C9218', 'C143655', 'C2560', 'C83502', 'C2587', 'C167106', 'C171533', 'C2645', 'C99945', 'C171511', 'C143031', 'C53851', 'C171509', 'C115835', 'C122850', 'C165219', 'C2612', 'C81603', 'C2647', 'C2926', 'C91103', 'C2705', 'C127863', 'C144919', 'C75907', 'C2703', 'C184182', 'C48742', 'C171512', 'C156308', 'C153222', 'C180375', 'C2602', 'C181092', 'C133603', 'C2667', 'C2608', 'C143650',

In [126]:
# print the first mosfet
# print(mosfets[0])

# sort by price, check if the stock is > 1000, if so, return that mosfet

# else, return the mosfet with the highest stock

# if no mosfet is found, return None

def get_best_mosfet(mosfets):
    # sort by price
    mosfets.sort(key=lambda x: x['Price'][0]['price'])
    # check if the stock is > 1000, if so, return that mosfet
    for mosfet in mosfets:
        if mosfet['Price'][0]['quantity'] > 1000:
            return mosfet
    # else, return the mosfet with the highest stock
    mosfets.sort(key=lambda x: x['Price'][0]['quantity'], reverse=True)
    return mosfets[0]

print(get_best_mosfet(mosfets))

IndexError: list index out of range

In [90]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Initialize the app with your service account
# cred = credentials.Certificate('path/to/your/serviceAccountKey.json')
# firebase_admin.initialize_app(cred)

# db = firestore.client()

# Define your field mappings here
field_mappings = {
    'current(A)': 'current_A',
    'drain_source_voltage(V)': 'drain_source_voltage_V',
    'on_resistance(Ω)': 'on_resistance_Ω'
}

# Function to recreate documents with updated field names
def recreate_documents(collection_name):
    # Fetch all documents in the collection
    docs = db.collection(collection_name).stream()
    
    for doc in docs:
        new_data = {}
        old_data = doc.to_dict()
        # Copy data to new document, replacing field names as necessary
        for old_field, value in old_data.items():
            new_field = field_mappings.get(old_field, old_field)  # Get new field name if it exists
            new_data[new_field] = value
        
        # Create a new document with the updated data
        new_doc_ref = db.collection(collection_name).document()  # You could also use a specific naming scheme
        new_doc_ref.set(new_data)
        
        # Delete the old document
        db.collection(collection_name).document(doc.id).delete()
        
        print(f"Recreated document {doc.id} as {new_doc_ref.id}")

# Recreate documents in the 'mosfets' collection
recreate_documents('mosfets')



Recreated document C105685 as NtYumvD706OZ2AyYLQGT
Recreated document C10644 as KimHOwvGR2IFQXuKD4kG
Recreated document C10647 as 3L9me36uGgPn1nsKabqH
Recreated document C10650 as TZeYacDpc78E0x9re4bM
Recreated document C10654 as NbCWFcbLdnY9cD42uiRv
Recreated document C10657 as OdSGoVDKius6vJHNoVia
Recreated document C10663 as 7NeeA6hNGXaIzyu5iJCd
Recreated document C10666 as f52bKhttfi0ybdsavS1J
Recreated document C10668 as nQTp5a08T7q26fkcKjc8
Recreated document C10669 as 5KdN7h7YNQmqEmAJYyq1
Recreated document C106992 as qnVYgKShY3xiRuDKG011
Recreated document C107589 as c9PqPTKocTYU8KlXsOn8
Recreated document C108891 as ea4amXCusuCTMoFxL70Q
Recreated document C109445 as Rjplbz4prX3xkjSqvBBi
Recreated document C110100 as lz88ota7Y4l1en6PZtF8
Recreated document C110101 as q4RgUVGL9wHMIYMe7DJq
Recreated document C110158 as zYLVyBcpN0iXThmRCNqh
Recreated document C110710 as Yw8H2UKuELTygVP3pGPp
Recreated document C110711 as Zm8psKNaDVkN34LjwTDo
Recreated document C110712 as Do5Dyz0uoy

In [98]:
# Function to rename documents using 'MFR.Part #' as the new document ID
def rename_documents(collection_name):
    # Fetch all documents in the collection
    docs = db.collection(collection_name).stream()

    for doc in docs:
        data = doc.to_dict()
        # Check if 'MFR.Part #' exists and is suitable for use as a document ID
        new_doc_id = data.get('JLCPCB Part #')
        if new_doc_id:
            # Create a new document with 'MFR.Part #' as the document ID and the same data
            new_doc_ref = db.collection(collection_name).document(new_doc_id)
            new_doc_ref.set(data)

            # Delete the old document if the new document ID is different
            if new_doc_id != doc.id:
                db.collection(collection_name).document(doc.id).delete()
                print(f"Renamed document {doc.id} to {new_doc_id}")
        else:
            print(f"Document {doc.id} does not have a valid 'MFR.Part #'")

# Rename documents in the 'mosfets' collection
rename_documents('mosfets')

Renamed document 02dTzsAhvbvld4RQe6vH to C2603
Renamed document 04uP6Nc3ruNXzNEgvRb7 to C169762
Renamed document 09Cm4OzC4ea3NcIEOcQG to C2604
Renamed document 09yMQmVvBqmlRVMwVkXK to C2936
Renamed document 0Dr0OVdrDYyNFxpGbnOO to C169759
Renamed document 0Evgf1rIXH5CvHlRJsRH to C169755
Renamed document 0HzpEWxxDarwzBCToWTm to C167149
Renamed document 0IfW3rxiO6woEv459x9P to C79666
Renamed document 0LiSG6U457J0Q3PfrIUJ to C7592
Renamed document 0SxbEenVtKPGk4mJ362G to C129166
Renamed document 0V5cNwocEN6rHijlMSzH to C115838
Renamed document 0Xx05VfRS2FmxgHMnK5L to C115828
Renamed document 0XztsAZrFQ2cVlvUhr80 to C3010
Renamed document 0e2V76DYiKS9FbnLMVMP to C167106
Renamed document 0eZEIaBStUYG9H23seat to C171555
Renamed document 0gAwRAWtdUBvZeDcEh8U to C143657
Renamed document 0sjxBttVERfT9OgnJvlA to C144954
Renamed document 0xQPoa04lzQ0jn1fwJM4 to C143706
Renamed document 0xk6keX9JvCDmVPKLYqT to C75882
Renamed document 14ibOeFgt8I8sEZ9jpsb to C165218
Renamed document 17ywQanQtv6dDEP

In [97]:
# Define your field mappings here
field_mappings = {
    'power_dissipation(W)': 'power_dissipation_W',  # Updated mapping
    'on_resistance_Ω' : 'on_resistance_ohms'
}

# Function to recreate documents with updated field names
def recreate_documents(collection_name):
    # Fetch all documents in the collection
    docs = db.collection(collection_name).stream()
    
    for doc in docs:
        new_data = {}
        old_data = doc.to_dict()
        # Copy data to new document, replacing field names as necessary
        for old_field, value in old_data.items():
            new_field = field_mappings.get(old_field, old_field)  # Get new field name if it exists
            new_data[new_field] = value
        
        # Create a new document with the updated data
        new_doc_ref = db.collection(collection_name).document()  # You could also use a specific naming scheme
        new_doc_ref.set(new_data)
        
        # Delete the old document
        db.collection(collection_name).document(doc.id).delete()
        
        print(f"Recreated document {doc.id} as {new_doc_ref.id}")

# Recreate documents in the 'mosfets' collection
recreate_documents('mosfets')

Recreated document 03f3KJVRtIFa6slW4BTE as m5yOyGcR556sEzPfO4qT
Recreated document 03mEUUGBDXYGvoFYavxq as qARCEziRtjkcCmfUy9Kw
Recreated document 0GoDXv65CVjRjK3deuPa as SM2TfI5OMJMzuYGLTzRL
Recreated document 0MqXpbuJvmBMWXPdgiUM as dkfwhtuCa1KtxULNZCXB
Recreated document 0TudmVzFZwqR7rB09pZs as z5Vzp0wSNnDtIFlGOgKN
Recreated document 0ny8fKwAfUbRPdmkx0Lh as Kq8Ew8Rwq5QQaLgz6p0i
Recreated document 0rdsP5YMGvryCm4X8njh as 3OgqbtB5OASvjFOTXIKU
Recreated document 0yXxZTUPNd8Uffx6jLcJ as JduyK7rI78olkjOXbSHS
Recreated document 14Ek6VW8mhAEcrXX9HLE as QqHyvalEUbQ6ArnmSkTV
Recreated document 19IFHmtGlSX9R9CKGvrM as kJFjqbpqSfspbNaDXtVB
Recreated document 1ADCp0y5ara858IkohuX as sbQF2m7QogpAj3H44s7X
Recreated document 1DX9UwH4UnbZXUuMmRxi as KeUB7lwJJq8kvo81mGr3
Recreated document 1G80c0xBmbdCrCRqPqN3 as csmhuZmTfZVywMUhZVaw
Recreated document 1bOJDkQOiFieg49Lpink as llLGRZsKLdk95hS2DFr7
Recreated document 1mCSkdT8qwBofWK0Vvgb as qfGhwXDxsamtfjJ6Sbtm
Recreated document 1qNsXlWOTm87XCJEHJVn 